In [1]:
pip install transformers==4.45.2 accelerate


   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 9.9/9.9 MB 61.7 MB/s  0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB ?  0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 68.4 MB/s  0:00:00

   ------ --------------------------------- 1/6 [regex]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggingface-hub]
   ------------- -------------------------- 2/6 [huggi

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

X_TRAIN_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\datachallenge_deep\X_train_uDRk9z9")
X_TEST_DIR  = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\datachallenge_deep\X_test_xNbnvIa")
X_UNLAB_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\datachallenge_deep\X_unlabeled_mtkxUlo")
Y_TRAIN_CSV = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\datachallenge_deep\Y_train_T9NrBYo.csv")

OUT_ROOT = Path("./runs")
OUT_ROOT.mkdir(exist_ok=True)

NUM_CLASSES = 3
IGNORE_INDEX = 255

# SegFormer 更稳：pad 到 W=288（160 本身是 32 的倍数）
H, W_PAD = 160, 288


In [ ]:
import re

NAME_RE = re.compile(r"well_(\d+)_section_(\d+)_patch_(\d+)$")
def parse_name(stem: str):
    m = NAME_RE.match(stem)
    if not m: raise ValueError(stem)
    return int(m.group(1)), int(m.group(2)), int(m.group(3))

def list_npy(dir_path: Path):
    files = sorted(list(dir_path.rglob("*.npy")))
    if not files:
        files = sorted(list(dir_path.rglob("*.NPY")))
    return files

def load_x_npy(path: Path) -> np.ndarray:
    x = np.load(path)
    if x.ndim == 3 and x.shape[0] == 1:
        x = x[0]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx > mn:
        x = (x - mn) / (mx - mn)
    else:
        x = np.zeros_like(x, dtype=np.float32)
    return x  # (160,w)

def pad_x_to_288(x: np.ndarray) -> np.ndarray:
    h, w = x.shape
    out = np.zeros((h, W_PAD), dtype=np.float32)
    out[:, :w] = x
    return out

y_df = pd.read_csv(Y_TRAIN_CSV, index_col=0)

def restore_mask_from_row(row_values: np.ndarray) -> np.ndarray:
    vals = row_values[row_values != -1]
    return vals.reshape(160, -1).astype(np.int64)  # (160,160) or (160,272)

def pad_mask_to_288(mask: np.ndarray, w: int) -> np.ndarray:
    out = np.full((H, W_PAD), IGNORE_INDEX, dtype=np.int64)
    out[:, :w] = mask
    return out

def make_valid_mask(w: int) -> np.ndarray:
    valid = np.zeros((H, W_PAD), dtype=np.bool_)
    valid[:, :w] = True
    return valid
